In [44]:
dataset_path = "data/crag_task_1_and_2_dev_v4.jsonl.bz2"
# dataset_path = "data/crag_task_3_dev_v4.tar.bz2"

In [45]:
# import tarfile
# import json
# import io
#
# with tarfile.open(dataset_path, 'r:bz2') as tar:
#     for member in tar.getmembers():
#         if member.name.endswith('.jsonl'):
#             f = tar.extractfile(member)
#             if f:
#                 # Use a buffered reader for efficiency
#                 reader = io.TextIOWrapper(f, encoding='utf-8')
#                 for line in reader:
#                     json_obj = json.loads(line)
#                     # Process your JSON object
#                     print(json_obj)  # Example: print each object
#                     break

In [46]:
import json
import bz2

questions = []
search_results = {}
duplicate_count = 0

with bz2.open(dataset_path, "rt") as file:
    i = 0
    for line in file:
        j_line = json.loads(line)
        # print(j_line.keys())
        # break
        questions.append({
            "id": j_line['interaction_id'],
            "question": j_line['query'],
            "golden_answers": [str(j_line['answer'])],
            "alt_ans": j_line['alt_ans'],
            "metadata": {
                "query_time": j_line['query_time'],
                "domain": j_line['domain'],
                "question_type": j_line['question_type'],
                "static_or_dynamic": j_line['static_or_dynamic'],
                "alt_ans": j_line['alt_ans'],
            }
        })
        for search_result in j_line['search_results']:
            # print(search_result.keys())
            # search_results.add(search_result)
            hsh = hash(frozenset(search_result.items()))
            if hsh in search_results:
                duplicate_count += 1
            else:
                search_results[hsh] = search_result
        # break
        # print(j_line.keys())
        # print("j_line")
        # print(j_line['query'])
        # for k, v in j_line.items():
        #     print(f"{k}: {v}")
        #     break
        # break
        # if i > 5:
        #     break
        # i += 1
    # data = json.load(file)

In [47]:
len(questions)

2706

In [48]:
len(search_results)

10713

In [49]:
duplicate_count

2817

In [50]:
# save questions
with open("flashrag_format_data/flashrag_crag_questions_str.jsonl", "w") as f:
    for question in questions:
        f.write(json.dumps(question) + "\n")

In [8]:
corpus_example_row = {
    "id": 38,
    "title": "What is the smallest mammal?",
    "contents": ""
}

In [19]:
# show single search result dictionary
page = search_results[7244461135126064771]
page_name = page['page_name']
page_url = page['page_url']
page_html = page['page_result']
page.keys()

dict_keys(['page_name', 'page_url', 'page_snippet', 'page_result', 'page_last_modified'])

In [34]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page_html, "lxml")
text = soup.get_text(" ", strip=True)  # Use space as a separator, strip whitespaces

In [28]:
# print(text)
text

"Travel + Leisure Co. (TNL) Dividend Date & History | Koyfin Skip to Content Product Features Data Coverage Data Coverage Use Cases Independent Investors Financial Advisors Traders Research Analyst Sales Students Enterprise Schools Pricing Sign Up for Free Log In Log In Sign Up for Free TNL Travel + Leisure Co. Travel + Leisure Co. TNL NYSE $44.87 USD 0.19 0.43% Market Cap $3,187.19M Forward P/E 8.4x Volume 416,329 Total Return (3M) 21.95% Travel + Leisure Co. ( TNL ) Dividend Date & History Sign Up for Free Dividend Data Travel + Leisure Co.’s ( TNL ) dividend yield is 4.46%, which means that for every\n        $100 invested in the company's stock, investors would receive $4.46 in dividends per year. Travel + Leisure Co.’s payout ratio is 34.34% which means that 34.34% of the company's\n        earnings are paid out as dividends. A low payout ratio may indicate that the company has a strong financial position\n        and can invest in growth opportunities, while a high payout ratio m

In [22]:
from blingfire import text_to_sentences_and_offsets

OSError: dlopen(/Users/jakubkucera-sch/Documents/diplomka/CRAG_v4dev/.venv/lib/python3.11/site-packages/blingfire/libblingfiretokdll.dylib, 0x0006): tried: '/Users/jakubkucera-sch/Documents/diplomka/CRAG_v4dev/.venv/lib/python3.11/site-packages/blingfire/libblingfiretokdll.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/jakubkucera-sch/Documents/diplomka/CRAG_v4dev/.venv/lib/python3.11/site-packages/blingfire/libblingfiretokdll.dylib' (no such file), '/Users/jakubkucera-sch/Documents/diplomka/CRAG_v4dev/.venv/lib/python3.11/site-packages/blingfire/libblingfiretokdll.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))

In [31]:
# import nltk
# nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /Users/jakubkucera-
[nltk_data]     sch/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [35]:
from nltk.tokenize import sent_tokenize
# text = "Hello world. This is a test."
sentences = sent_tokenize(text)
# ["Hello world.", "This is a test."]

In [37]:
len(sentences)

46

In [38]:
len(text)

7911

In [40]:
unchunked_corpus = []
chunked_corpus = []

In [41]:
global_counter = 0
for c, search_result in enumerate(search_results.values()):
    page_html = search_result['page_result']
    soup = BeautifulSoup(page_html, "lxml")
    text = soup.get_text(" ", strip=True)  # Use space as a separator, strip whitespaces
    unchunked_corpus.append({
        "id": c,
        "title": search_result['page_name'],
        "contents": text
    })
    sentences = sent_tokenize(text)
    for sentence in sentences:
        chunked_corpus.append({
            "id": global_counter,
            "title": search_result['page_name'],
            "contents": sentence
        })
        global_counter += 1

/var/folders/gm/6zjcwbd164d1v4gmx73009pr0000gp/T/ipykernel_50619/2361309978.py:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(page_html, "lxml")


In [42]:
with open("flashrag_format_data/flashrag_crag_unchunked_corpus.jsonl", "w") as f:
    for item in unchunked_corpus:
        f.write(json.dumps(item) + "\n")

In [43]:
with open("flashrag_format_data/flashrag_crag_chunked_corpus.jsonl", "w") as f:
    for item in chunked_corpus:
        f.write(json.dumps(item) + "\n")